# Лабораторная работа №5: Машины опорных векторов  

*Модели*: PCR, Случайный лес, kNN

*Данные*: `winequality-white_for_lab` (источник: https://github.com/ania607/ML/blob/main/data/winequality-white_for_lab.csv)

# Загружаем пакеты

In [192]:
# загрузка пакетов: инструменты -------------------------------------------- 
#  работа с массивами

import numpy as np 
#  фреймы данных
import pandas as pd 
#  для таймера 
import time 
 
# загрузка пакетов: модели ------------------------------------------------- 
#  логистическая регрессия
from sklearn.linear_model import LogisticRegression 
# стандартизация
from sklearn.preprocessing import StandardScaler 
# метод главных компонент 
from sklearn.decomposition import PCA 
# конвейеры
from sklearn.pipeline import make_pipeline 
# перекрёстная проверка и метод проверочной выборки 
from sklearn.model_selection import cross_val_score
# для перекрёстной проверки и сеточного поиска 
from sklearn.model_selection import KFold, GridSearchCV 
#  сводка по точности классификации 
from sklearn.metrics import classification_report 
# случайный лес
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
# kNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

In [193]:
# константы
#  ядро для генератора случайных чисел
my_seed = 5

## Загружаем данные

`winequality-white_for_lab` – преобразованные наборы данных со статистикой по качеству белых вин. Источник: сайт Калифорнийского университета в Ирвине.
Набор в формате csv доступен по ссылке: https://raw.githubusercontent.com/ania607/ML/main/data/winequality-white_for_lab.csv

## Список столбцов файла:   

* `fixed_acidity` – постоянная кислотность;  
* `volatile_acidity` – переменная кислотность;  
* `citric_acid` – содержание лимонной кислоты;  
* `residual_sugar` – остаточный сахар;  
* `chlorides` – содержание хлоридов;  
* `free_sulfur_dioxide` – содержание диоксида серы в свободном виде;  
* `total_sulfur_dioxide` – общее содержание диоксида серы;  
* `density` – плотность;  
* `pH` – кислотность; 
* `sulphates` – содержание сульфатов;  
* `alcohol` – содержание алкоголя; 
* `quality` – балльная оценка качества вина, от 0 до 10;  
* `Y` – целевая переменная: 1 = высокое качество (`quality` > 5), 0 = низкое (`quality` <= 5).    

Загружаем данные во фрейм и выясняем их размерность.

In [194]:
# загружаем таблицу и превращаем её во фрейм
fileURL = 'https://raw.githubusercontent.com/ania607/ML/main/data/winequality-white_for_lab.csv'
DF_raw = pd.read_csv(fileURL)
DF_raw = DF_raw.drop(['quality'], axis=1)

# выясняем размерность фрейма
print('Число строк и столбцов в наборе данных:\n', DF_raw.shape)

Число строк и столбцов в наборе данных:
 (4898, 12)


In [195]:
# первые 5 строк
DF_raw.head(5)

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,Y
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,1
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,1
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,1
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,1


In [196]:
# типы столбцов фрейма
DF_raw.dtypes

fixed_acidity           float64
volatile_acidity        float64
citric_acid             float64
residual_sugar          float64
chlorides               float64
free_sulfur_dioxide     float64
total_sulfur_dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
Y                         int64
dtype: object

In [197]:
# считаем пропуски в каждом столбце
DF_raw.isna().sum()

fixed_acidity           0
volatile_acidity        0
citric_acid             0
residual_sugar          0
chlorides               0
free_sulfur_dioxide     0
total_sulfur_dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
Y                       0
dtype: int64

Пропусков не обнаружено

Отложим 15% наблюдений для прогноза.

In [198]:
# наблюдения для моделирования
DF = DF_raw.sample(frac=0.85, random_state=my_seed)
# отложенные наблюдения
DF_predict = DF_raw.drop(DF.index)

In [199]:
DF.describe()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,Y
count,4163.000000,4163.000000,4163.000000,4163.000000,4163.000000,4163.00000,4163.000000,4163.000000,4163.000000,4163.000000,4163.000000,4163.000000
mean,6.853291,0.279188,0.335191,6.432573,0.045988,35.15950,138.579750,0.994063,3.187785,0.490106,10.504111,0.663944
std,0.848668,0.101255,0.122810,5.114131,0.022455,16.64706,42.027749,0.003015,0.151869,0.113494,1.234149,0.472415
min,3.800000,0.080000,0.000000,0.600000,0.009000,2.00000,9.000000,0.987130,2.720000,0.220000,8.000000,0.000000
25%,6.300000,0.210000,0.270000,1.700000,0.036000,23.00000,109.000000,0.991760,3.080000,0.410000,9.400000,0.000000
50%,6.800000,0.260000,0.320000,5.200000,0.043000,33.00000,134.000000,0.993780,3.180000,0.480000,10.400000,1.000000
75%,7.300000,0.320000,0.390000,10.000000,0.050000,46.00000,167.500000,0.996200,3.280000,0.550000,11.400000,1.000000
max,14.200000,1.005000,1.660000,65.800000,0.346000,146.50000,366.500000,1.038980,3.820000,1.080000,14.200000,1.000000


# Преобразование исходных данных и построение моделей
## Стандартизация и переход к главным компонентам

Предварительно преобразуем пространство исходных показателей с помощью метода главных компонент

In [200]:
# стандартизация
sc = StandardScaler()
X_train_std = sc.fit_transform(DF.iloc[:, :12].values)

# проверяем средние и стандартные отклонения после стандартизации
for i_col in range(X_train_std.shape[1]) :
    print('Столбец ', i_col, ': среднее = ',
          np.round(np.mean(X_train_std[:, i_col]), 2),
         '   Станд. отклонение = ', 
          np.round(np.std(X_train_std[:, i_col]), 2), sep='')

Столбец 0: среднее = 0.0   Станд. отклонение = 1.0
Столбец 1: среднее = -0.0   Станд. отклонение = 1.0
Столбец 2: среднее = 0.0   Станд. отклонение = 1.0
Столбец 3: среднее = 0.0   Станд. отклонение = 1.0
Столбец 4: среднее = -0.0   Станд. отклонение = 1.0
Столбец 5: среднее = -0.0   Станд. отклонение = 1.0
Столбец 6: среднее = 0.0   Станд. отклонение = 1.0
Столбец 7: среднее = 0.0   Станд. отклонение = 1.0
Столбец 8: среднее = -0.0   Станд. отклонение = 1.0
Столбец 9: среднее = 0.0   Станд. отклонение = 1.0
Столбец 10: среднее = -0.0   Станд. отклонение = 1.0
Столбец 11: среднее = -0.0   Станд. отклонение = 1.0


## Регрессия на главные компоненты (PCR)  

Пересчитаем объясняющие показатели в главные компоненты.  

In [201]:
# функция с методом главных компонент
pca = PCA()
# пересчитываем в главные компоненты (ГК)
X_train_pca = pca.fit_transform(X_train_std)

# считаем доли объяснённой дисперсии
frac_var_expl = pca.explained_variance_ratio_
print('Доли объяснённой дисперсии по компонентам в PLS:\n',
     np.around(frac_var_expl, 3),
     '\nОбщая сумма долей:', np.around(sum(frac_var_expl), 3))

Доли объяснённой дисперсии по компонентам в PLS:
 [0.277 0.131 0.116 0.089 0.081 0.078 0.065 0.06  0.047 0.03  0.024 0.002] 
Общая сумма долей: 1.0


Первые две главные компоненты объясняют 41.4% разброса 11 объясняющих переменных

In [202]:
# данные для обучения моделей
X_train = DF.iloc[:, :11] 
y_train = DF.iloc[:, -1]

# объединяем в конвейер шкалирование, ГК с 2 компонентами и логит
pipe_lr = make_pipeline(StandardScaler(),
                        PCA(n_components = 2),
                        LogisticRegression(random_state = my_seed, 
                                           solver = 'lbfgs'))

# будем сохранять точность моделей в один массив
score = list()
score_models = list()

# считаем точность с перекрёстной проверкой, показатель Acc
cv = cross_val_score(estimator = pipe_lr, X = X_train, y = y_train, 
                     cv = 5, scoring='accuracy')

# записываем точность
score.append(np.around(np.mean(cv), 3)) 
score_models.append('pca_logit')

print('Acc с перекрёстной проверкой',
      '\nдля модели', score_models[0], ':', score[0])

Acc с перекрёстной проверкой 
для модели pca_logit : 0.66


# Случайный лес  

У модели случайного леса два настроечных параметра: количество деревьев $B$ и количество признаков для построения отдельного дерева $m$. Настроим сеточный поиск для их подбора.  

In [203]:
X = DF.drop(['Y'], axis=1)
y = DF['Y']

# сколько столбцов в обучающих данных (p)
X_m = X.shape[1]
# возьмём значения для m: p, p/2, sqrt(p) и log2(p)
ms = np.around([X_m, X_m / 2, np.sqrt(X_m), np.log2(X_m)]).astype(int)
ms

array([11,  6,  3,  3])

In [204]:
# разбиения для перекрёстной проверки
kfold = KFold(n_splits=10, random_state=my_seed, shuffle=True)

# настроим параметры случайного леса с помощью сеточного поиска
param_grid = {'n_estimators' : list(range(10, 51, 2)),
              'max_features' : ms}

# таймер
tic = time.perf_counter()
clf = GridSearchCV(RandomForestClassifier(DecisionTreeClassifier()),
                   param_grid, scoring='accuracy', cv=kfold)
random_forest = clf.fit(X, y)
# таймер
toc = time.perf_counter()
print(f"Сеточный поиск занял {toc - tic:0.2f} секунд", sep='')

Сеточный поиск занял 326.39 секунд


In [205]:
# точность лучшей модели
np.around(random_forest.best_score_, 3)

0.837

In [206]:
# записываем точность
score.append(np.around(random_forest.best_score_, 3))
score_models.append('random_forest_GS')

print('Acc с перекрёстной проверкой',
      '\nдля модели', score_models[1], ':', score[1])

Acc с перекрёстной проверкой 
для модели random_forest_GS : 0.837


# Метод kNN

Реализуем метод k-ближайших соседей с преобразованием PCA.

In [207]:
k_range = list(range(1,50))
weight_options = ["uniform", "distance"]

pipe_knn = Pipeline(steps=[('standard', StandardScaler()), ('PCA', PCA(n_components = 10)), 
                           ('knn', KNeighborsClassifier())])
param_grid = {'knn__n_neighbors': list(range(1,50))}

knn = KNeighborsClassifier()
kfold = KFold(n_splits = 10, random_state = my_seed, shuffle = True)
grid = GridSearchCV(pipe_knn, param_grid, cv=kfold, scoring='accuracy')

tic = time.perf_counter()
grid.fit(X_train,y_train)
toc = time.perf_counter()
print(f"Сеточный поиск занял {toc - tic:0.2f} секунд", sep='')

Сеточный поиск занял 49.48 секунд


In [208]:
score.append(np.around(grid.best_score_,3))
score_models.append('sc_pca_knn')

print('Acc с перекрёстной проверкой','\nдля модели',score_models[2],':',score[2])

Acc с перекрёстной проверкой 
для модели sc_pca_knn : 0.794



# Прогноз на отложенные наблюдения по лучшей модели

Ещё раз посмотрим на точность построенных моделей. 

In [209]:
# сводка по точности моделей
pd.DataFrame({'Модель' : score_models, 'Acc' : score})

,Модель,Acc
0,pca_logit,0.660
1,random_forest_GS,0.837
2,sc_pca_knn,0.794


Все модели показывают хорошую точность по показателю $Acc$, при этом самой точной оказывается модель PCA + SVC. Сделаем прогноз на отложенные наблюдения.   

In [210]:
# данные для прогноза
X_pred = DF_predict.drop(['Y'], axis=1)
# строим прогноз
y_hat = random_forest.best_estimator_.predict(X_pred)
# характеристики точности
print(classification_report(DF_predict['Y'], y_hat))

              precision    recall  f1-score   support

           0       0.75      0.71      0.73       241
           1       0.86      0.88      0.87       494

    accuracy                           0.83       735
   macro avg       0.81      0.80      0.80       735
weighted avg       0.83      0.83      0.83       735

